# My Playground for Neural Network

In [217]:
from utils import *
from stratified import *
from run import *
from neuralnetwork import *

In [218]:
def importhousedata():
    house = importfile('hw3_house_votes_84.csv', ',')
    housecategory = {}
    for i in house[0]:
        housecategory[i] = 'categorical'
    housecategory["class"] = 'class'
    housedata = np.array(house[1:]).astype(float)
    return housedata, housecategory
def importwinedata():
    wine = importfile('hw3_wine.csv', '\t')
    winecategory = {}
    for i in wine[0]:
        winecategory[i] = 'numerical'
    winecategory["# class"] = 'class'
    winedata = np.array(wine[1:]).astype(float)
    return winedata, winecategory
def importcmcdata():
    cmc = importfile('cmc.data', ',')
    cmccategory = {"Wife's age":"numerical","Wife's education":"categorical",
    "Husband's education":"categorical","Number of children ever born":"numerical",
    "Wife's religion":"binary","Wife's now working?":"binary",
    "Husband's occupation":"categorical","Standard-of-living index":"categorical",
    "Media exposure":"binary","Contraceptive method used":"class"}
    cmcdata = np.array(cmc).astype(int)
    return cmcdata, cmccategory
def importcancerdata():
    cancer = importfile('hw3_cancer.csv', '\t')
    cancercategory = {}
    for i in cancer[0]:
        cancercategory[i] = 'numerical'
    cancercategory["Class"] = 'class'
    cancerdata = np.array(cancer[1:]).astype(float)
    return cancerdata, cancercategory

housedata, housecategory = importhousedata()
winedata, winecategory = importwinedata()
cmcdata,cmccategory = importcmcdata()
cancerdata, cancercategory = importcancerdata()
example_category = {'ece': 'numerical','cs': 'categorical','phy': 'categorical','math': 'categorical','musi': 'categorical','grade': 'class'}
#example_data = np.array([[100,0,1,1,0,0],[123,1,0,1,1,1],[44,1,0,0,0,2],[92,1,0,1,0,3],[73,0,1,0,0,2]])
example_data = np.array([[10,0,1,1,0,0],[23,1,0,1,1,1],[44,1,0,0,0,2],[92,1,0,1,0,3],[73,0,1,0,0,2],[122,0,1,1,0,0],[124,1,0,1,1,1],[44,1,0,0,0,2],[40,1,0,1,0,3],[73,0,1,0,0,2]])

In [219]:
ohe_house_data, ohe_house_category = onehotencoder(housedata, housecategory)
ohe_wine_data, ohe_wine_category = onehotencoder(winedata,winecategory)
ohe_cmc_data, ohe_cmc_category = onehotencoder(cmcdata,cmccategory)
ohe_example_data, ohe_example_category = onehotencoder(example_data,example_category)
ohe_cancer_data, ohe_cancer_category = onehotencoder(cancerdata,cancercategory)

In [220]:
n_ohe_house_data = normalizetrain(ohe_house_data,ohe_house_category)[0]
n_ohe_wine_data = normalizetrain(ohe_wine_data,ohe_wine_category)[0]
n_ohe_cmc_data = normalizetrain(ohe_cmc_data,ohe_cmc_category)[0]
n_ohe_example_data = normalizetrain(ohe_example_data,ohe_example_category)[0]
n_ohe_cancer_data = normalizetrain(ohe_cancer_data,ohe_cancer_category)[0]

In [221]:
def transposelistoflist(l):
    newlistoflist = []
    for i in range(len(l[0])):
        newlist = []
        for j in range(len(l)):
            newlist.append(l[j][i])
        newlistoflist.append(newlist)
    return newlistoflist

In [222]:
def gradientD(weights_list,deltalist,attributelist,biasterm=True):
    gradlist = []
    for i in range(len(weights_list)):
        attributenow = attributelist[i]
        deltanow = np.array([deltalist[i]]).T
        dotproduct = deltanow*attributenow
        # print('dotshape',dotproduct.shape)
        gradlist.append(dotproduct)
    return gradlist

In [223]:
def g(x): # sigmoid function
    return 1/(1 + np.exp(-x))

In [224]:
# Forward propagation vectorized
def neural_network(normed_hotted_data,ohe_category,weights_list,biasterm=True, minibatchk = 15, lambda_reg = 0.1, learning_rate = 0.01):
    normed_ohe_copy = normed_hotted_data.copy()
    if minibatchk > len(normed_hotted_data):
        minibatchk = len(normed_hotted_data)
    np.random.shuffle(normed_ohe_copy)
    splitted = np.array_split(normed_ohe_copy, minibatchk)
    
    inputcategory, outputcategory = [],[]
    inputindex, outputindex = [],[]
    n = 0
    for i in ohe_category:
        if ohe_category[i] != 'class_numerical':
            inputcategory.append(i) # name of the input category
            inputindex.append(n) # index of the input category
        else:
            outputcategory.append(i) # name of the output category
            outputindex.append(n) # index of the output category
        n += 1
    
    b = 1 if biasterm else 0
    
    for onebatch in splitted:
        onebatch = onebatch.T
        input_data = onebatch[inputindex].T
        output_data = onebatch[outputindex].T
        # input_data_mean = onebatch[inputindex].mean(axis=1)
        output_data = onebatch[outputindex].T

        # forward propagation
        instance_index = 0
        j = 0
        listofgradient = []
        for one_instance in input_data:
            current_layer_a = np.append(1,one_instance) if b == 1 else one_instance
            # input layer is the current layer
            current_layer_index = 0
            output_expect = output_data[instance_index]
            attributesnobias = [one_instance]
            attributeswbias = [current_layer_a]
            for theta in weights_list:
                z = np.dot(theta,current_layer_a)
                a = g(z)
                current_layer_a = np.append(1,a) if (b == 1) and (current_layer_index+1 != len(weights_list)) else a
                attributesnobias.append(a)
                attributeswbias.append(current_layer_a)
                current_layer_index += 1

            output_predict = current_layer_a # the last attribute is the output for this batch.
            instance_index += 1
            j += costfunction(output_expect,output_predict)

            # calculate delta blame (back propagation)
            listofdelta = blame(output_predict,output_expect,weights_list, attributeswbias)
            thisgradient = gradientD(weights_list,listofdelta,attributeswbias,biasterm)
            listofgradient.append(thisgradient)
        
        gradientP = [lambda_reg*t for t in weights_list]
        # first column in singleP in the np.array = 0
        for singleP in gradientP:
            singleP[:, 0] = 0
        
        grad_D_transpose = transposelistoflist(listofgradient)
        grad_D_sum = [np.sum(t,axis=0) for t in grad_D_transpose]
        gradients_batch = []
        for i in range(len(grad_D_sum)):
            gradients_batch.append((grad_D_sum[i] + gradientP[i])*(1/instance_index))
        
        j /= (instance_index+1)
        s = sumofweights(weights_list,bias=b)*lambda_reg/(2*(instance_index+1))
        allj = j+s # total cose with regularization

        # update weights
        for i in range(len(weights_list)):
            weights_list[i] -= learning_rate*gradients_batch[i]

    return weights_list, allj, j

In [225]:
w = np.array([[1,2,3],[4,5,6]])
w[:, 0] = 0
w

array([[0, 2, 3],
       [0, 5, 6]])

In [226]:
normalizeinuse = n_ohe_wine_data
categoryinuse = ohe_wine_category

In [227]:
layerdataparameter = [5,5,5] # [First layer, Second layer, Third layer], not including the bias and input & output layer
initweights = initialize_weights(categoryinuse,layerdataparameter)
kk,jj,pj = neural_network(normalizeinuse,categoryinuse,initweights)

In [228]:
j_previous = 99
epsilon = 80
weight, j_now, purej = neural_network(normalizeinuse,categoryinuse,initweights)
epsilon = j_previous - j_now
j_previous = j_now
smallestj = j_now
smallestpure = purej
count = 0
while (epsilon > 0.00001 or j_now >= smallestj or purej >= smallestpure) and (count < 10000 or purej >= smallestpure):
    print(count)
    count+=1
    weight, j_now, purej = neural_network(normalizeinuse,categoryinuse,weight)
    print('purej',purej)
    print('j now is',j_now)
    smallestj = min(smallestj,j_now)
    smallestpure = min(smallestpure,purej)
    epsilon = abs(j_previous - j_now)
    print(epsilon)
    j_previous = j_now

0
purej 2.461270640321009
j now is 2.611412595542679
0.7971037615992369
1
purej 2.5658914241295947
j now is 2.7188130393461454
0.10740044380346658
2
purej 2.5219210991830607
j now is 2.678567182373574
0.040245856972571215
3
purej 2.983484618505915
j now is 3.1450425102642536
0.46647532789067947
4
purej 3.103115443239709
j now is 3.2714540039026803
0.12641149363842663
5
purej 3.6562533492872404
j now is 3.8328454670817536
0.5613914631790733
6
purej 3.2785266331990877
j now is 3.4656845970789902
0.3671608700027633
7
purej 3.7371298477353627
j now is 3.936744383951439
0.47105978687244887
8
purej 3.1499041318199517
j now is 3.364184013151006
0.5725603708004332
9
purej 3.8213948171947965
j now is 4.05261527994087
0.688431266789864
10
purej 3.3744460630565656
j now is 3.6244689261812297
0.42814635375964016
11
purej 5.737905063127446
j now is 6.007651697335847
2.383182771154617
12
purej 5.8324915931652335
j now is 6.123228029550462
0.11557633221461483
13
purej 5.362427450655221
j now is 5.674

/Users/harrywanghc/Developer/2022/COMPSCI589/COMPSCI589-HW4/neuralnetwork.py:32: RuntimeWarning: divide by zero encountered in log
  j = -np.dot(expected_output,np.log(actual_output)) - np.dot((1 - expected_output),np.log(1 - actual_output))


In [229]:
print(j_now)
correctweight = weight

nan


In [230]:
def traverse_network(input, weight):
    for i in range(len(weight)):
        input = np.append(1,input) if i != (len(weight)) else input
        z = np.dot(weight[i],input)
        a = g(z)
        input = a
    output = a
    n = 0
    for item in output:
        if item< 0.5:
            output[n] = 0
        else:
            output[n] = 1
        n += 1
    return output

In [231]:
def predictusingnetwork(data, category, weight):
    n = 0
    inputindex, outputindex = [],[]
    for i in category:
        if category[i] != 'class_numerical':
            inputindex.append(n)
        else:
            outputindex.append(n)
        n += 1
    predictoutput = traverse_network(data[inputindex],weight)
    realoutput = data[outputindex]
    # print(predictoutput, realoutput)
    return predictoutput, realoutput

In [232]:
allcount = 0
correctcount = 0
for instance in normalizeinuse:
    predict, real = predictusingnetwork(instance,categoryinuse,correctweight)
    for i in range(len(predict)):
        if predict[i] == real[i]:
            correctcount += 1
        allcount += 1
    
print(correctcount/allcount)

0.40074906367041196
